# Метод, основанный на правилах и словарях

Для определения эмоциональной окраски была создана обучающая и тестовая выборка. На основе обучающей выборки будет создан тезаурус эмоционально окрашенной лексики. Отзывы обучающей выборки не присутствуют в тестовой.

**Описание данных:**

- _text_ - отзыв пользователя
- _rated_ - оценка пользователя
- _coloring_ - эмоциональная окраска (определена вручную на основе оценок)
- _date_ - дата размещения отзыва
- _bank_ - банк, о котором написан отзыв
- _theme_ - тема обращения
- _name_ - имя пользователя
- _sex_ - пол пользователя (определен вручную на основе имени)


**Обучающая выборка**

    Исходный датафрейм содержит отзывы о банках с первых 100 страниц веб-сайта Сравни.ру. Размер корпуса - 4116 отзывов.
    Для составления словарей были выбраны отзывы с оценками '1' и '2' - для негативно окрашенной лексики (1329 отзыва), '5' - для позитивно окрашенной лексики (1357 отзыва).
    
    
**Тестовая выборка**

    Для проведения анализа были собраны отзывы со страниц 130-160. Размер корпуса - 696 отзывов. Корпус содержит отзывы с оценками 1-5 и метаданные, описанные выше.
    
**Ход исследования:**

1. [Импорт библиотек](#1)
2. [Чтение и обзор файлов](#2)
3. [Составление тональных словарей](#3)
4. [Анализ тональности](#4)
5. [Оценка работы алгоритма](#5)
6. [Обнаруженные закономерности](#6)
7. [Результаты](#7)




## Импорт необходимых библиотек <a id="1"></a>

In [2]:
import pandas as pd
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from matplotlib import pyplot as plt
import numpy as np

## Чтение файлов <a id="2"></a>

Загрузим корпус для проведения будущего анализа.

In [3]:
df = pd.read_csv('/Users/js/Desktop/disser/testtt_corpus.csv')
df = df[df['text'].notna()]
df = df.sort_values(by='text')
df = df.drop(df.columns[[0]], axis = 1)

In [4]:
df.head(10)

,text,rated,coloring,date,bank,theme,name,sex
243,А где реквизиты то взять???,2,bad,"12 нояб, 2014",zapadnyj,Кредиты наличными,Яна,female
266,А как стоило открыть в Вашем банке счет – так ...,2,bad,"30 сент, 2014",rnkb,Обслуживание,Ленара,unkn
142,Банк Траст в июне 2014 году выдал справку о за...,1,bad,"08 нояб, 2017",trast,Кредиты наличными,Елена,female
625,Банк всегда готов помочь клиентам в решение пр...,5,good,"02 апр, 2022",sovkombank,Вклады,Елена,female
373,"Банк нравится, готова и дальше сотрудничать. П...",5,good,"05 апр, 2022",sovkombank,Ипотека,Альфия,unkn
54,"Банк обещает ""4 мили за каждые 100 рублей"" при...",1,bad,"01 мар, 2018",vtb,Дебетовые карты,Наталья,female
213,Банк предложил мне кредит наличными на погашен...,1,bad,"20 авг, 2017",vtb,Кредиты наличными,Елена,female
207,Банк предложил рефинансирование кредита под ма...,1,bad,"28 авг, 2017",vtb,Кредиты наличными,Дмитрий,male
163,"Банк пройдоха еще тот, сейчас буду подавать в ...",1,bad,"17 окт, 2017",vtb,Ипотека,Инна,female
177,"Банк соответствует своему названию - Совковый,...",1,bad,"06 окт, 2017",sovkombank,Вклады,NaN,unkn


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 696 entries, 243 to 42
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      696 non-null    object
 1   rated     696 non-null    int64 
 2   coloring  696 non-null    object
 3   date      696 non-null    object
 4   bank      696 non-null    object
 5   theme     693 non-null    object
 6   name      639 non-null    object
 7   sex       696 non-null    object
dtypes: int64(1), object(7)
memory usage: 48.9+ KB


Корпус <b>отрицательных</b> отзывов:

In [7]:
df_neg = pd.read_csv('/Users/js/neg_corp.csv')
df_neg = df_neg.drop(df_neg.columns[[0]], axis = 1)

In [8]:
df_neg.head(10)

,text,rated,coloring,date,name,bank,sex
0,Возмутительный способ обмана придумали крючкот...,1,bad,"04 апр, 2022",Сергей,vtb,male
1,"Не храните в нем деньги, ни коем случае! У мен...",1,bad,"30 мар, 2022",Альбина,uralsib,female
2,С 09.03.22 не могу получить средства с евро сч...,1,bad,"24 мар, 2022",Ирина,uralsib,female
3,"Очень долгое обслуживание, сотрудники совершен...",1,bad,"14 мар, 2022",Александр,gazprombank,male
4,Звоню третий день по вопросу валюты. Оператор ...,1,bad,"14 мар, 2022",Сергей Владимирович,gazprombank,unkn
5,"Опыт сложился негативный, Сегодня 3марта 2022 ...",1,bad,"03 мар, 2022",Илья,gazprombank,male
6,и 01.03.2022 приложения не работают. в 23ч при...,1,bad,"01 мар, 2022",Марина,gazprombank,female
7,ОАО Газпромбанк офис г. Новосибирск ул. Богдан...,1,bad,"01 мар, 2022",Виктория,gazprombank,female
8,Мы с мамой обратились 24.02.2022 в отделение Г...,1,bad,"26 февр, 2022",алена,gazprombank,unkn
9,Омск. С 24.02.22 невозможно снять наличные ден...,1,bad,"26 февр, 2022",Олег,gazprombank,male


In [9]:
df_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329 entries, 0 to 1328
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1329 non-null   object
 1   rated     1329 non-null   int64 
 2   coloring  1329 non-null   object
 3   date      1329 non-null   object
 4   name      1227 non-null   object
 5   bank      1329 non-null   object
 6   sex       1329 non-null   object
dtypes: int64(1), object(6)
memory usage: 72.8+ KB


Корпус <b>положительных</b> отзывов:

In [10]:
df_pos = pd.read_csv('/Users/js/pos_corp.csv')
df_pos = df_pos.drop(df_pos.columns[[0]], axis = 1)

In [11]:
df_pos.head(10)

,text,rated,coloring,date,name,bank,sex
0,"Недавно Газпромбанк обновил приложение, и это ...",5,good,"15 апр, 2022",Константин,gazprombank,male
1,Сменил работу и зарплатный банк соответственно...,5,good,"14 апр, 2022",Глеб,gazprombank,male
2,В Газпромбанке я получаю пенсию по потере корм...,5,good,"14 апр, 2022",NaN,gazprombank,unkn
3,По совету сестры заказала себе дебетовую карту...,5,good,"14 апр, 2022",Лили,gazprombank,unkn
4,"Доброго времени суток, рассказываю об автокред...",5,good,"14 апр, 2022",NaN,gazprombank,unkn
5,Устроилась на первую серьезную работу после ун...,5,good,"14 апр, 2022",Алена,gazprombank,unkn
6,Открыл ИП через Точка Банк в 2019г. При этом н...,5,good,"13 апр, 2022",Антон,tochka,male
7,Пол года назад я смог рефинансировать свою ипо...,5,good,"13 апр, 2022",Аркадий,gazprombank,male
8,Хочу выразить благодарность ГазПромБанку за ка...,5,good,"13 апр, 2022",NaN,gazprombank,unkn
9,Привет зарплатникам Газпромбанка! В приложении...,5,good,"13 апр, 2022",NaN,gazprombank,unkn


In [12]:
df_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1357 entries, 0 to 1356
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1357 non-null   object
 1   rated     1357 non-null   int64 
 2   coloring  1357 non-null   object
 3   date      1357 non-null   object
 4   name      1301 non-null   object
 5   bank      1357 non-null   object
 6   sex       1357 non-null   object
dtypes: int64(1), object(6)
memory usage: 74.3+ KB


## Составление тональных словарей <a id="3"></a>

Определим части речи и коллокации, которые войдут в словарь:

In [13]:
# позитивно и негативно окрашенные прилагательные, существительные, глаголы и наречия
pos_adj = {} 
pos_noun = {}
pos_verb = {}
pos_adv = {}
neg_adj = {}
neg_noun = {}
neg_verb = {}
neg_adv = {}
# позитивно и негативно окрашенные коллокации
# прилагательное+существительное, наречие+глагол,
# глагол+существительное косвенного падежа, существительное именительного падежа+глагол
pos_adj_noun = {}
neg_adj_noun= {} 
pos_adv_verb= {} 
neg_adv_verb = {} 
pos_verb_noun = {}
neg_verb_noun= {}
pos_noun_verb = {}
neg_noun_verb = {}

Создадим списки слов и частоту их встречаемости по корпусу с помощью функций:

In [14]:
def word_list(corpus,tone):
    
    '''
    Списки отдельных слов. 
    
    Функция принимает на вход датафрейм и нужную тональность (+/-),
    токенизирует каждый отзыв датафрейма, для каждого слова отзыва проводит морфологический анализ,
    далее заполняет словари по тегу части речи.
   
    '''
    
    
    m = MorphAnalyzer()
   
    for comment in corpus:
        for w in simple_word_tokenize(comment):
            w = m.parse(w)[0]
            n = w.normal_form

            my_dict={}
            if 'ADJF' in w.tag:
                if tone=='-':
                    my_dict=neg_adj
                else:
                    my_dict=pos_adj

            if 'NOUN' in w.tag:
                if tone=='-':
                    my_dict=neg_noun
                else:
                    my_dict=pos_noun

            if 'VERB' in w.tag:
                if tone=='-':
                    my_dict=neg_verb
                else:
                    my_dict=pos_verb

            if 'ADVB' in w.tag:
                if tone=='-':
                    my_dict=neg_adv
                else:
                    my_dict=pos_adv

            if n not in my_dict:
                my_dict[n]=1
            else:
                my_dict[n]+=1 

In [15]:
def colloc_list(corpus,tone):
    
    ''' 
    Списки коллокаций. 
    
    Функция принимает на вход датафрейм и нужную тональность (+/-),
    токенизирует каждый отзыв датафрейма, для каждого слова отзыва проводит морфологический анализ,
    далее заполняет словари по тегу части речи слова и тегу слова, следующего за ним.
   
    '''
    
    m = MorphAnalyzer()
    
    for comment in corpus:
        comment = simple_word_tokenize(comment)
        for i in range (0, len(comment)-2):
            w1 = comment[i]
            w2 = comment[i+1]
            w3 = comment[i+2]
            w1 = m.parse(w1)[0]
            n1 = w1.normal_form
            w2 = m.parse(w2)[0]
            n2 = w2.normal_form
            w3 = m.parse(w3)[0]
            
            my_dict={}
            
            if ('ADJF' in w1.tag and 'NOUN' in w2.tag) or ('ADJF' in w2.tag and 'NOUN' in w1.tag):
                if (w1.tag.gender == w2.tag.gender) and (w1.tag.number == w2.tag.number) and (w1.tag.case == w2.tag.case):
                    if tone=='-':
                        my_dict=neg_adj_noun
                    else:
                        my_dict=pos_adj_noun
                        
                    n = n1+' '+n2           
                    if n not in my_dict:
                        my_dict[n]=1
                    else:
                        my_dict[n]+=1 
                        
                        
            if ('ADVB' in w1.tag and 'VERB' in w2.tag) or ('ADVB' in w2.tag and 'VERB' in w1.tag):
                if tone=='-':
                    my_dict=neg_adv_verb
                else:
                    my_dict=pos_adv_verb
                    
                n = n1+' '+n2           
                if n not in my_dict:
                    my_dict[n]=1
                
                else:
                    my_dict[n]+=1 
                          
                
            if 'VERB' in w1.tag and 'PREP' in w2.tag and 'NOUN' in w3.tag:
                if tone=='-':
                    my_dict=neg_verb_noun
                else:
                    my_dict=pos_verb_noun
                    
                n = n1+' '+n2+' '+comment[i+2]         
                if n not in my_dict:
                    my_dict[n]=1
                else:
                    my_dict[n]+=1 
                    
                    
            if 'NOUN' in w1.tag and 'VERB' in w2.tag:
                if w1.tag.case == 'nomn':
                    if tone=='-':
                        my_dict=neg_noun_verb
                    else:
                        my_dict=pos_noun_verb
            
                    n = comment[i]+' '+comment[i+1]  
                    if n not in my_dict:
                        my_dict[n]=1
                
                    else:
                        my_dict[n]+=1 

In [16]:
colloc_list(df_neg['text'],tone='-')

In [17]:
colloc_list(df_pos['text'],tone='+')

In [18]:
word_list(df_neg['text'],tone='-')

In [19]:
word_list(df_pos['text'],tone='+')

Создадим два общих списка коллокаций:

In [20]:

pos = {**pos_adj_noun,**pos_adv_verb,**pos_verb_noun,**pos_noun_verb}
neg = {**neg_adj_noun,**neg_adv_verb,**neg_verb_noun,**neg_noun_verb}


In [26]:
print('neg_adj', neg_adj)


neg_adj {'возмутительный': 2, 'годовой': 49, 'свой': 653, 'минимальный': 25, 'первый': 193, 'самый': 140, 'накопительный': 46, 'некий': 16, 'который': 908, 'соседний': 14, 'основный': 21, 'открытый': 24, 'мой': 1209, 'такой': 594, 'кой': 9, 'новый': 188, 'любой': 81, 'этот': 806, 'всякий': 20, 'тот': 673, 'активный': 5, 'ручной': 1, 'каждый': 180, 'полный': 152, 'долгий': 27, 'третий': 71, 'профильный': 1, 'негативный': 24, 'технический': 84, 'четвёртый': 9, 'сей': 102, 'какой': 327, 'личный': 207, 'денежный': 183, 'другой': 474, 'хмельницкий': 1, 'ипотечный': 103, 'трёхкратный': 1, 'данный': 257, 'кредитный': 449, 'опознавательный': 1, 'рабочий': 108, 'готовый': 6, 'повышенный': 19, 'ваш': 313, 'весь': 453, 'интересный': 29, 'маленький': 30, 'сам': 306, 'служебный': 6, 'хороший': 55, 'дисциплинарный': 3, 'учётный': 7, 'мошеннический': 25, 'учёткий': 2, 'стандартный': 13, 'подводный': 7, 'льготный': 19, 'наличный': 54, 'должностной': 4, 'горячий': 370, 'административный': 2, 'уголовный

Вычислим критерий согласия Пирсона:

In [27]:
def x_sq (dict1, dict2):
    
    '''
    Функция присваивает значения переменных для формулы хи-квадрат и вычисляет критерий согласия,
    далее добавляет слово и значение х2 в словарь, если частота лексемы в первом словаре больше, чем во втором 
    и возвращает отсортированный словарь.
    
    '''
    
    total1 = sum(dict1.values())
    total2 = sum(dict2.values())

    dict_sorted={}

    for word in dict1:
        a = dict1[word]
        b = 0
        if word in dict2:
            b = dict2[word]
            c = total1 - a
            d = total2 - b
            n = a+b+c+d
            x2 = (n*((a*d-b*c)**2))/((a+b)*(a+c)*(b+d)*(c+d))
            
            if a>b:
                dict_sorted[word] = x2

    l = lambda x: x[1]
    dict_sorted = sorted(dict_sorted.items(), key=l, reverse=True)
    return dict_sorted

In [28]:
a = x_sq(pos_adj, neg_adj)
aa = x_sq(neg_adj, pos_adj)
b = x_sq(pos_noun, neg_noun)
bb = x_sq(neg_noun, pos_noun)
c = x_sq(pos_verb, neg_verb)
cc = x_sq(neg_verb, pos_verb)
d = x_sq(pos_adv, neg_adv)
dd = x_sq(neg_adv, pos_adv)
e = x_sq(pos, neg)
ee = x_sq(neg, pos)

In [29]:
print(a, '\n\n', aa,'\n\n',b,'\n\n',bb,'\n\n',c,'\n\n',cc,'\n\n',d,'\n\n',dd,'\n\n',e,'\n\n',ee)

[('хороший', 789.0949478821026), ('удобный', 721.0179193086933), ('отличный', 434.51449478067633), ('выгодный', 312.4076418082152), ('вежливый', 309.37573212774976), ('приятный', 198.7122426413362), ('высокий', 138.53570608900577), ('быстрый', 131.66264772193344), ('заёмный', 117.96478907483966), ('бонусный', 110.35989855390115), ('понятный', 90.1389656520824), ('внимательный', 79.03059561513636), ('отзывчивый', 79.03059561513636), ('оперативный', 70.09534575721503), ('большой', 64.04796102139775), ('грамотный', 60.05813217727102), ('реальный', 59.75390932669219), ('собственный', 58.62554717579247), ('интересный', 57.11105326332213), ('знакомый', 54.98464181198011), ('положительный', 50.010655780350675), ('приветливый', 46.85478650547974), ('доброжелательный', 43.3735752394401), ('доступный', 42.59227514144516), ('супер', 39.82021540147462), ('оптимальный', 38.94107905272273), ('коммунальный', 38.94107905272273), ('бесплатный', 35.55212241980939), ('прекрасный', 35.09351032704231), ('н




Далее подберем оптимальное значение <b>критерия согласия Пирсона</b> для добавления слов в тезаурус.
Преобразуем словари в датафреймы и определим первый квартиль, это значение будет пороговым для каждого словаря.




In [32]:
dict_list = [a,aa,b,bb,c,cc,d,dd,e,ee]
threshold = []
for i in dict_list:
    i = pd.DataFrame(i, columns=['w','x'])
    n = np.percentile(i.x, 25)
    threshold.append(n)

In [33]:
threshold

[12.507576828621886,
 0.12214436242938595,
 8.299354268914863,
 0.20298328889912964,
 6.772234142264276,
 0.23180454537703205,
 5.894387272924615,
 0.17212468657163785,
 4.587118202929423,
 0.08529805806329623]

In [34]:
pos_adj = []
for key,value in a:
    if value > 12.5:
        pos_adj.append(key)
    
neg_adj = []
for key,value in aa:
    if value > 0.1:
        
        neg_adj.append(key)
    
pos_noun = []
for key,value in b:
    if value > 8.3:
        
        pos_noun.append(key)
    
neg_noun = []
for key,value in bb:
    if value > 0.2:
        
        neg_noun.append(key)
        
pos_adv = []
for key,value in d:
    if value > 6.8:
        
        pos_adv.append(key)
    
neg_adv = []
for key,value in dd:
    if value > 0.2:
        
        neg_adv.append(key)
    
pos_verb = []
for key,value in c:
    if value > 0.09:
        
        pos_verb.append(key)
    
neg_verb = []
for key,value in cc:
    if value > 1.2:
        
        neg_verb.append(key)
    
pos_colloc = []
for key,value in e:
    if value > 4.6:
        
        pos_colloc.append(key)

neg_colloc = []
for key,value in ee:
    if value > 0.09:
        
        neg_colloc.append(key)


In [35]:
print('pos_adj', pos_adj, '\n\n', 'pos_noun', pos_noun,'\n\n', 'pos_verb', pos_verb,'\n\n', 'pos_adv', pos_adv,'\n\n', 'neg_adj', neg_adj,'\n\n', 'neg_noun', neg_noun,'\n\n', 'neg_verb', neg_verb,'\n\n', 'neg_adv', neg_adv,'\n\n', 'pos_colloc', pos_colloc,'\n\n', 'neg_colloc', neg_colloc)

pos_adj ['хороший', 'удобный', 'отличный', 'выгодный', 'вежливый', 'приятный', 'высокий', 'быстрый', 'заёмный', 'бонусный', 'понятный', 'внимательный', 'отзывчивый', 'оперативный', 'большой', 'грамотный', 'реальный', 'собственный', 'интересный', 'знакомый', 'положительный', 'приветливый', 'доброжелательный', 'доступный', 'супер', 'оптимальный', 'коммунальный', 'бесплатный', 'прекрасный', 'неплохой', 'лёгкий', 'адекватный', 'умный', 'сложный', 'качественный', 'трудный', 'различный', 'беспроцентный', 'приличный', 'короткий', 'полезный', 'приемлемый', 'продуктовый', 'чёткий', 'достойный', 'привлекательный', 'крутой', 'прозрачный'] 

 pos_noun ['халва', 'рассрочка', 'партнёр', 'покупка', 'кэшбэк', 'бонус', 'кэшбек', 'магазин', 'кешбек', 'остаток', 'кешбэк', 'акция', 'товар', 'использование', 'десяток', 'персонал', 'магазин-партнёр', 'скидка', 'пользование', 'подписка', 'кэш', 'нарекание', 'уровень', 'доктор', 'задание', 'преимущество', 'интерфейс', 'рада', 'выполнение', 'балл', 'переплата'

## Анализ тональности <a id="4"></a>

Для каждого комментария вычислим значение `score`, которое будет отражать количество использованной положительно и отрицательно окрашенной лексики.

In [38]:
def analisys(corpus):
    
    '''
    Функция принимает на вход датафрейм, токенизирует каждый отзыв датафрейма, 
    для каждого слова отзыва проводит морфологический анализ,
    далее, если это слово или коллокация есть в тональном словаре, 
    прибавляет или вычитает единицу к изначально нулевому значению score 
    (в зависимости от тональности слова и наличия частицы 'не').
    Заполняет список значениями score для каждого отзыва.
   
    '''
    m = MorphAnalyzer()
    hhh = []
    for comment in corpus:
        comment = simple_word_tokenize(comment)
        score = 0
        for i in range (0, len(comment)-2):
            w1 = comment[i]
            w2 = comment[i+1]
            w3 = comment[i+2]
            w1 = m.parse(w1)[0]
            n1 = w1.normal_form
            w2 = m.parse(w2)[0]
            n2 = w2.normal_form
            w3 = m.parse(w3)[0]
            n = n1+' '+n2 
            nn = n1+' '+n2+' '+comment[i+2]
        
        
            if ('ADJF' in w1.tag and 'NOUN' in w2.tag) or ('ADJF' in w2.tag and 'NOUN' in w1.tag):
                if (w1.tag.gender == w2.tag.gender) and (w1.tag.number == w2.tag.number) and (w1.tag.case == w2.tag.case):
                    
                    if n in pos_colloc:
                        score += 1
                    elif n in neg_colloc:
                        score += -1
                    else:
                        score += 0                     
            
            elif 'ADJF' in w2.tag:
                if n2 in pos_adj and w1 == 'не':
                    score += -1
                elif n2 in pos_adj and w1 != 'не':
                    score += 1
                elif n2 in neg_adj:
                    score += -1
                else:
                    score += 0
                
            elif 'NOUN' in w1.tag and 'VERB' in w2.tag:
                if nn in pos_colloc:
                    score += 1
                elif n in neg_colloc:
                    score += -1
                else:
                    score += 0 
        
            elif 'NOUN' in w2.tag:
                if n2 in pos_noun and w1 == 'не':
                    score += -1
                elif n2 in neg_noun and w1 == 'не':
                    score += 1
                elif n2 in neg_noun:
                    score += -1
                else:
                    score += 0
        
            elif 'VERB' in w1.tag and 'PREP' in w2.tag and 'NOUN' in w3.tag:
                if nn in pos_colloc:
                    score += 1
                elif n in neg_colloc:
                    score += -1
                else:
                    score += 0 
        
            elif 'VERB' in w2.tag:
                if n2 in pos_verb and w1 == 'не':
                    score += -1
                elif n2 in neg_verb and w1 == 'не':
                    score += 1
                elif n2 in neg_verb:
                    score += -1
                else:
                    score += 0
        
            elif ('ADVB' in w1.tag and 'VERB' in w2.tag) or ('ADVB' in w2.tag and 'VERB' in w1.tag):
                if n in pos_colloc:
                    score += 1
                elif n in neg_colloc:
                    score += -1
                else:
                    score += 0
                
            elif 'ADVB' in w2.tag:
                if n2 in pos_adv and w1 == 'не':
                    score += -1
                elif n2 in neg_adv and w1 == 'не':
                    score += 1
                elif n2 in neg_adv:
                    score += -1
                else:
                    score += 0
                
            else:
                score += 0
        
            
        
        hhh.append(score)
        
    return hhh

In [39]:
score = analisys(df['text'])

Добавим значения в датафрейм и заменим колонку `coloring` на `true`, которая будет отражать реальную оценку.

In [40]:
df['score'] = score

In [41]:
sent = []
for i in df['coloring']:
    if i == 'good':
        sent.append('pos')
    else:
        sent.append('neg')
df = df.drop(['coloring'], axis=1)
df['true'] = sent

Проведем анализ тональности для корпусов с положительными и отрицательными отзывами, чтобы определить пороговые значения `score` для предсказания оценки.

In [42]:
bad = analisys(df_neg['text'])
good = analisys(df_pos['text'])

In [43]:
bad = pd.DataFrame(bad, columns=['score'])
good = pd.DataFrame(good, columns=['score'])

In [44]:
bad.describe()

,score
count,1329.000000
mean,-49.147479
std,40.936635
min,-206.000000
25%,-65.000000
50%,-35.000000
75%,-20.000000
max,-3.000000


In [45]:
good.describe()

,score
count,1357.000000
mean,-6.727340
std,8.181889
min,-133.000000
25%,-8.000000
50%,-5.000000
75%,-3.000000
max,4.000000


Нормальные значения `score` для отрицательных отзывов от -71 до -21, для положительных от -8 до 4.
Присвоим отзывам со значением score меньше или равному `-15` отрицательную оценку, остальным - положительную.

In [46]:
pred = []
for i in df['score']:
    if i <= -15:
        pred.append('neg')
    else:
        pred.append('pos')
df['predicted'] = pred

In [47]:
df

,text,rated,date,bank,theme,name,sex,score,true,predicted
243,А где реквизиты то взять???,2,"12 нояб, 2014",zapadnyj,Кредиты наличными,Яна,female,-2,neg,pos
266,А как стоило открыть в Вашем банке счет – так ...,2,"30 сент, 2014",rnkb,Обслуживание,Ленара,unkn,-25,neg,neg
142,Банк Траст в июне 2014 году выдал справку о за...,1,"08 нояб, 2017",trast,Кредиты наличными,Елена,female,-31,neg,neg
625,Банк всегда готов помочь клиентам в решение пр...,5,"02 апр, 2022",sovkombank,Вклады,Елена,female,-4,pos,pos
373,"Банк нравится, готова и дальше сотрудничать. П...",5,"05 апр, 2022",sovkombank,Ипотека,Альфия,unkn,-4,pos,pos
...,...,...,...,...,...,...,...,...,...,...
287,"я ответила, что была по девичьей фамилии, по и...",2,"21 июня, 2014",lipeckkombank,Обслуживание,Ал,unkn,-74,neg,neg
269,я пришел брать одобренный мне кредит на сумму ...,2,"03 сент, 2014",promsvjazbank,Обслуживание,Руслан,male,-94,neg,neg
65,"я являюсь клиентом этого банка, т.к зарплату м...",1,"12 февр, 2018",chelindbank,Кредиты наличными,Яна,female,-32,neg,neg
67,января 2018 года в 18:44 в СПб на Нарвском 23 ...,1,"08 февр, 2018",sovkombank,Обслуживание,Вера,female,-14,neg,pos


## Оценка работы алгоритма <a id="5"></a>

Для оценки качества работы будем использовать метрики `accuracy`, `f-measure`, `f1-score`, `precision` и `recall`.

Посчитаем количество `TP` (*true positive*), `TN` (*true negative*), `FP` (*false positive*), `FN` (*false negative*).

In [48]:
a = []
b = []
for i in df['predicted']:
    a.append(i)
for i in df['true']:
    b.append(i)

In [49]:
tp = 0 
tn = 0 
fp = 0 
fn = 0
t_f = []
for i in range(0, len(a)):
    if a[i] == b[i]:
        if a[i] == 'pos':
            tp+=1
            t_f.append('tp')
        else:
            tn+=1
            t_f.append('tn')
    elif a[i] != b[i]:
        if a[i] == 'pos':
            fp+=1
            t_f.append('fp')
        else:
            fn+=1
            t_f.append('fn')
        

In [50]:
df['result'] = t_f

Вычислим метрики по формулам.

In [51]:
accuracy = (tp+tn)/len(df)
precision = tp/ (tp+fp)
recall = tp/(tp+fn)
f1 = 2/(1/precision+1/recall)
f_measure = 2*((precision*recall)/(precision+recall))

In [52]:
print('accuracy', accuracy)
print('precision', precision)
print('recall', recall)
print('f1-score', f1)
print('f-measure', f_measure)

accuracy 0.9267241379310345
precision 0.9198966408268734
recall 0.9468085106382979
f1-score 0.9331585845347314
f-measure 0.9331585845347313


In [53]:
df.head(10)

,text,rated,date,bank,theme,name,sex,score,true,predicted,result
243,А где реквизиты то взять???,2,"12 нояб, 2014",zapadnyj,Кредиты наличными,Яна,female,-2,neg,pos,fp
266,А как стоило открыть в Вашем банке счет – так ...,2,"30 сент, 2014",rnkb,Обслуживание,Ленара,unkn,-25,neg,neg,tn
142,Банк Траст в июне 2014 году выдал справку о за...,1,"08 нояб, 2017",trast,Кредиты наличными,Елена,female,-31,neg,neg,tn
625,Банк всегда готов помочь клиентам в решение пр...,5,"02 апр, 2022",sovkombank,Вклады,Елена,female,-4,pos,pos,tp
373,"Банк нравится, готова и дальше сотрудничать. П...",5,"05 апр, 2022",sovkombank,Ипотека,Альфия,unkn,-4,pos,pos,tp
54,"Банк обещает ""4 мили за каждые 100 рублей"" при...",1,"01 мар, 2018",vtb,Дебетовые карты,Наталья,female,-67,neg,neg,tn
213,Банк предложил мне кредит наличными на погашен...,1,"20 авг, 2017",vtb,Кредиты наличными,Елена,female,-41,neg,neg,tn
207,Банк предложил рефинансирование кредита под ма...,1,"28 авг, 2017",vtb,Кредиты наличными,Дмитрий,male,-14,neg,pos,fp
163,"Банк пройдоха еще тот, сейчас буду подавать в ...",1,"17 окт, 2017",vtb,Ипотека,Инна,female,-57,neg,neg,tn
177,"Банк соответствует своему названию - Совковый,...",1,"06 окт, 2017",sovkombank,Вклады,NaN,unkn,-35,neg,neg,tn


# Закономерности <a id="6"></a>

Посмотрим на отзывы, где алгоритм ошибочно определил оценку.

In [54]:
t_res = df.query('result == "tn" or result == "tp"')
f_res = df.query('result == "fn" or result == "fp"')

In [55]:
pd.options.display.max_colwidth = 10000
f_res = f_res.drop(['date', 'bank', 'theme', 'name', 'sex'], axis=1)

In [56]:
print(fp, fn)

31 20


In [66]:
falsep = f_res.query('result == "fp"')
o = []
for i in df['text']:
    o.append(len(i))
pd.DataFrame(o).describe()

,0
count,696.00000
mean,581.54023
std,703.10123
min,27.00000
25%,125.75000
50%,235.00000
75%,788.25000
max,3758.00000


Ложноположительные отзывы имеют тенденцию быть довольно короткими по сравнению с другими отзывами выборки. Был выявлен максимальный размер отзыва в выборке ложноположительных, он составляет 416 слов, при этом среднее значение длины отзывов в исходном тестовом корпусе – 703, медианное значение – 581. 

На основании этого можно сделать вывод, что алгоритм лучше проявит себя при наличии большего количества текстов, содержащих более 580 слов, и наоборот, при оценке тональности более коротких текстов, результаты будут ниже. При этом, стоит отметить, что алгоритм определял отзыв как негативный при значении ‘score’ < -15. При определенных задачах заказчика, в которых необходимо выявить все отрицательные отзывы, порог может быть изменен до максимального.


In [44]:
f_res.query('result == "fn"')

,text,rated,score,true,predicted,result
721,"Банком доволен на сто процентов , очень нравится оперативность решения вопросов через чат поддержку, последний раз столкнулся со случайным переводом заёмных средств на на другую карту , обратился в чат банка с просьбой отменить комиссию , так как деньги были свои на счету и платеж с кредитного счета был ошибочным , после нескольких минут ожиданий оператор отменила комиссию",5,-19,pos,neg,fn
406,"Быстро решили проблему с картой. Была утерена карта и проблемы со снятием своих денег. Менеджер выдала новую карту, объяснила изменения в тарифе. Помогла увеличить лимит снятия. Было нужно снять крупную сумму.",5,-16,pos,neg,fn
509,"В банке была проблема с п.о. и баланс карты отображался не верно, соответственно воспользоваться своими деньгами Я не мог около 2х часов пока банк исправлял ошибку. После мне прислали промокод на 1000 рублей за доставленные неудобства.",5,-15,pos,neg,fn
403,"В связи с определенными обстоятельствами очень боюсь хранить деньги дома, тк. сумма приличная, поэтому начала рассматривать варианты вкладов. В итоге остановилась на накопительном счете «Трать и копи», сумма не важна (там вроде от 1 р) и процент меня устраивает. Девушка – менеджер помогла выбрать и рассчитала такую мини табличку доходности вклада. Хочу сказать, что доход ощутимый и лишним точно не будет. Я решила, что этот вариант оптимальный. Хранить дома не вариант, а тут еще и какой-никакой, а пассивный доход.",5,-32,pos,neg,fn
395,"В целом пользованием банка доволен, есть подводные камни, но как говорится, халявный сыр)) Кредитная карта в полное прозрачна и меня полностью устраивает А вот кредит без процентов интересная штука, советую перед тем, как брать почитать в интернете, слава богу так и сделал) да в любом случае, процент получается не большой, вернее даже, чем у конкурентов, а ещё вернее не процент, а что-то типо страховки, будьте аккуратнее, читайте документы и отзывы и тогда вы будете довольным этим банком",5,-22,pos,neg,fn
526,"Вот уже как 2 года являюсь пользователем карты халва, спросите меня как я им стал, да очень просто, как и все мне понадобился небольшой кредит наличными. Совкомбанк предложил самые лучшие условия кредитования. Уже сейчас когда я хорошо знаю как работает программа 5х10, копится кешбек и растёт мой уровень я уверен что больше никогда не возьму кредит в другом банке. Карта халва это прекрасный продукт для тех кто привык извлекать выгоду из своих трат, с этой картой я трачу деньги с удовольствием.",5,-26,pos,neg,fn
720,"Делал ремонт на даче, не учел повышение цен и не хватило на материалы и оплату рабочих. Решился взять кредит в Газпромбанке, потому что в других банках был сумасшедший процент, ГПБ тоже поднял, но в этом банке я уже обслуживаюсь 3 года и меня все устраивает. Зашел на их сайт и сразу увидел кредитный калькулятор, по которому самому можно подобрать сумму, срок погашения и процент. Там же отправил заявку, ответ пришел в обед следующего дня, положительный. Сходил в офис , забрал и сразу же расплатился со строителями. Оплачивать кредит буду через приложение, чтобы не ездить каждый раз и не стоять в очередях",5,-27,pos,neg,fn
549,"Доброго дня. Я являюсь постоянным клиентом Совкомбанка. Так получилось, что я потеряла карту. Так как карта была с бесконтактной оплатой, мне пришлось её тут же заблокировать. На карте были все денежные средства. Сотрудники банка вошли в моё положение и быстро решили мой вопрос положительно. Я очень благодарна Совкомбанку и его сотрудникам за всегда внимательное и квалифицированное отношение к посетителям и всегда рекомендую своим друзьям и знакомым пользоваться услугами этого банка.",5,-20,pos,neg,fn
404,"Не понимаю, почему все возмущаются, что им страховку в банке предлагают, менталитет наш, наверное, ни копейки не переплатят, даже если это вопрос собственной безопасности. Я вот взяла потребкредит в Газпромбанке, и пока нет претензий к банку. Сотрудники оформили без лишней волокиты,и да, я сначала немного удивилась, что предложили оформи

В ложноотрицательных отзывах пользователи часто описывают определенные проблемы, которые банк помог решить, либо сравнивают один банк с другим. Из-за этого слов и коллокаций с отрицательной тональностью в тексте больше, и алгоритм определяет его как негативный.

В подобных отзывах слов и коллокаций с отрицательной тональностью в тексте больше, чем с положительной, и алгоритм определяет его как негативный. Точность алгоритма можно повысить добавлением триграмм в созданный тезаурус и учетом веса важности тональных слов в вычислении значения ‘score’ при проведении анализа. Эти задачи были оставлены для будущих работ.

In [45]:
pivot_bank = df.pivot_table(index = 'bank', values = 'score', aggfunc = 'mean').sort_values('score', ascending = False)
print('\nБанки, отзывы о которых содержат наименьшее количество негативной лексики:\n')
print(pivot_bank.head(10))



Банки, отзывы о которых содержат наименьшее количество негативной лексики:

                        score
bank                         
cmrbank             -6.000000
sovkombank          -8.525253
mezhtopehnergobank  -9.000000
zapadnyj           -13.000000
russtrojbank       -13.000000
venec              -15.000000
sovetskij          -16.000000
akibank            -17.000000
akcept             -18.000000
probiznesbank      -18.000000


In [46]:
print('\nБанки, отзывы о которых содержат наибольшее количество негативной лексики:\n')
print(pivot_bank.tail(10))



Банки, отзывы о которых содержат наибольшее количество негативной лексики:

                score
bank                 
jugra           -67.0
citibank        -74.5
pervomajskij    -79.5
tatfondbank     -94.0
krajinvestbank  -98.0
primore        -105.0
baltinvestbank -115.0
finam          -132.0
otkrytie       -138.0
siab           -140.0


In [47]:
pivot_theme = df.pivot_table(index = 'theme', values = 'score', aggfunc = 'mean').sort_values('score', ascending = False)
pivot_theme


,score
theme,
Рефинансирование ипотеки,-5.000000
Дистанционное обслуживание,-7.023810
Кредитные карты,-10.835366
Обмен валют,-16.000000
Другое,-16.761905
Вклады,-23.769231
Дебетовые карты,-23.871795
Автокредиты,-31.384615
Обслуживание,-35.530612


Видим, что рефинансирование ипотеки и дистанционное обслуживание вызывает у пользователей меньше негативных эмоций, чем вопросы связанные с обслуживанием в отделении и кредитом, что ожидаемо.

In [50]:
pivot_sex = df.pivot_table(index = 'sex', values = 'score', aggfunc = 'mean').sort_values('score', ascending = False)
pivot_sex


,score
sex,
female,-24.033835
male,-27.062780
unkn,-29.845411


In [52]:
len(df.query('sex== "male"'))

223

In [53]:
len(df.query('sex== "female"'))

266

## Общий вывод <a id="7"></a>

В ходе работы мы получили следующие результаты:

- Был создан тезаурус эмоционально окрашенной лексики на тему банков.


- Были созданы правила для проведения анализа тональности отзывов.


- Алгоритм хорошо справился с задачей. При оценке качества работы использовались метрики accuracy, f-measure, f1-score, precision и recall. Получены следующие значения:


    accuracy 0.93
    precision 0.92
    recall 0.95
    f1-score 0.93
    f-measure 0.93


- В ложноотрицательных отзывах пользователи часто описывают определенные проблемы, которые банк помог решить, либо сравнивают один банк с другим. Из-за этого слов и коллокаций с отрицательной тональностью в тексте больше, и алгоритм определяет его как негативный.


- В ложноположительных отзывах заметили, что все они довольно короткие. Алгоритму не хватило общего количества слов в отзыве, для правильного определения оценки. 


- Наименьшее количество негативной лексики содержат отзывы о банках: СМП-Банк, Совкомбанк, Межтопэнергобанк. Наибольшее количество - отзывы о банках: Югра, Ситибанк, Первомайский.


- Рефинансирование ипотеки и дистанционное обслуживание вызывает у пользователей меньше негативных эмоций, чем вопросы связанные с обслуживанием в отделении и кредитом.